In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
import math
import matplotlib.pyplot as plt


def plot_samples(data, n=5, max_cols=5, cmap="tab20"):
    n = min(len(data), n)
    if n < max_cols:
        rows = 1
        cols = n
    else:
        rows = math.ceil(n / max_cols)
        cols = max_cols
    print(f"rows: {rows}, cols: {cols}")
    fig, ax = plt.subplots(nrows=rows, ncols=cols, sharex=True, sharey=True, figsize=(20, 10))
    #ax = ax.flatten()
    index = 0
    for i in range(rows):
        for j in range(cols):
            img = data[index]
            index += 1
            #    .numpy().reshape(X.shape[1],X.shape[2],2)
            print()
            ax[i, j].imshow(img, cmap=cmap, interpolation='nearest')
            if index >= n:
                break

    #ax[0].set_xticks([])
    #ax[0].set_yticks([])
    plt.tight_layout()
    # plt.savefig('./figures/mnist_all.png', dpi=300)
    plt.show()

In [6]:
# Herunterladen der MNIST Daten als Trainings- und Testdaten
(mnist_images, mnist_labels), (mnist_images_test, mnist_labels_test) = tf.keras.datasets.mnist.load_data()

# Preprocessing der 
X_train = tf.cast(mnist_images[..., tf.newaxis] / 255, tf.float32)
y_train = tf.cast(mnist_labels, tf.int64)

X_test = tf.cast(mnist_images_test[..., tf.newaxis] / 255, tf.float32)
y_test = tf.cast(mnist_labels_test, tf.int64)

num_category = 10

y_train_v = tf.keras.utils.to_categorical(y_train, num_category)
y_test_v = tf.keras.utils.to_categorical(y_test, num_category)

dataset = tf.data.Dataset.from_tensor_slices(
    (tf.cast(mnist_images[..., tf.newaxis] / 255, tf.float32), tf.cast(mnist_labels, tf.int64)))
batch_size = 128
dataset = dataset.shuffle(1000).batch(batch_size)

dataset_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))
dataset_test = dataset_test.shuffle(1000).batch(batch_size)

x_train, x_test = mnist_images / 255.0, mnist_images_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")
batch = 128
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size=batch)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [8]:
mnist_images.shape

(60000, 28, 28)

In [ ]:
base_model = tf.keras.Sequential(
        [
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(20, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(10, activation="softmax", kernel_initializer="he_normal"),
        ]
    )
base_model.build(input_shape=(None, 28, 28, 1))
opti = tf.keras.optimizers.SGD(
        learning_rate=0.02, momentum=0.00,
        nesterov=False, name='SGD'
        )
opti2 = tf.keras.optimizers.Adelta()
opti3 = tf.keras.optimizers.Adam()
